In [6]:
import tess_stitch as tea
import big_data_funs as bd
import os
import matplotlib.pyplot as plt
from importlib import reload
from pathlib import Path
import numpy as np
reload(tea)
reload(bd);

In [7]:
# --- CONFIGURACIÓN ---
METODO = "LCF"
PCA_COMPONENTS = 5 

In [ ]:

df = bd.cargar_datos_NEA("big_data")

# print(df.head())

long = len(df)

for i in np.arange(long):
    TARGET = df['st_host'][i] 
    lc_stitched = tea.get_multisector_lc(
        TARGET, 
        method=METODO, 
        pca_components=PCA_COMPONENTS
    )
    if lc_stitched is not None:
        
        lc_clean = tea.clean_lc(lc_stitched, window_length=2001, sigma=5)
    
        star_data = tea.get_stellar_params(TARGET) 
        
        # Un poco overkill incorporando el data frame con la información de la estrella pero así sigue la misma lógica que tess_tea_stitch_single.py
        # De todas formas, la informaicón de la estrella que obtiene NEA es de TIC, así que a fin de cuentas es o mismo.
        
    else:
        print("No se pudieron obtener datos para el análisis.")
        continue

    if star_data is None:
        print("No se pudieron obtener datos para el análisis")
        continue
        

    folder = f"resultados_{TARGET}_stitching"
    SAVE_PATH = Path("resultados") / folder
    SAVE_PATH.mkdir(parents=True, exist_ok=True)

    
    # Cambia min_p y max_p para afinar qué periodo quieres observar
    bls_results = tea.bls(lc_clean, min_p=0.5, max_p=20, n_points=100000)
    
    # Planet name para guardar mejor los datos y también tenerlo en el título del gráfico
    planet_name = f"{df['pl_name'][i]}"
    
    params = tea.compute_params(bls_results, star_data)


    # En caso de que quieras ver los 3 plots generados para cada uno de los ~480 planetas que hay en el data frame, podrías descomentar esto (tardaría más tiempo)
    # tea.plots(lc_clean, bls_results, target_name=TARGET, save_path=None, planet_name = planet_name, method = METODO) 

    data_export = tea.pack_results_NEA(TARGET, planet_name, bls_results, star_data, params, df)

    tea.save_json(data_export, SAVE_PATH)